### Introduction

Each bag contains a collection of molecular fragments. The label is the sum of some property calculated for each fragment. These properties are chosen to be preferably additive (can be calculated as a linear contribution of each fragment). The predicted weights should correspond to the contribution of each fragment. The higher the contribution, the higher the weight should be.

**Instance:** One molecular fragment (e.g. C(=O)O - Carboxy group).

**Bag**: A collection of fragments (e.g., a list of 5 fragments).

**Label:** A sum of fragment property value (e.g. LogP) in the bag.

**Key instance:** All fragments.

In [ ]:
import logging
import warnings
warnings.filterwarnings("ignore")
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
logging.getLogger("lightning").setLevel(logging.ERROR)
from rdkit import RDLogger
# Suppress all RDKit warnings
RDLogger.DisableLog('rdApp.*')

import time
import torch
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# Preprocessing
from milearn.preprocessing import BagMinMaxScaler

# Network hparams
from milearn.network.module.hopt import DEFAULT_PARAM_GRID

# MIL wrappers
from milearn.network.regressor import BagWrapperMLPNetworkRegressor, InstanceWrapperMLPNetworkRegressor
from milearn.network.classifier import BagWrapperMLPNetworkClassifier, InstanceWrapperMLPNetworkClassifier

# MIL networks
from milearn.network.regressor import (InstanceNetworkRegressor,
                                       BagNetworkRegressor,
                                       AdditiveAttentionNetworkRegressor,
                                       SelfAttentionNetworkRegressor,
                                       HopfieldAttentionNetworkRegressor,
                                       DynamicPoolingNetworkRegressor)

# Utils
from scipy.stats import spearmanr
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import train_test_split

# Molecular utils
from qsarmil.data.fragment import create_fragment_bags
from rdkit import Chem
from rdkit.Chem import AllChem

# Prediction visualisation
from qsarmil.data.fragment import display_fragments_with_weights

# Data
from huggingface_hub import hf_hub_download

### Key Instance Detection Ranking Accuracy for Regression

This function evaluates how well a model's predicted attention weights rank the important instances in a bag, by computing the Spearman rank correlation between:

* The true importance ranking (represented here by the fragment property values)

* The predicted importance scores (predicted weights)

In [ ]:
def kid_ranking_accuracy(instance_digits, attn_weights):

    per_bag_corrs = []
    for w, digits in zip(attn_weights, instance_digits):
        if len(set(digits)) == 1:
            # Avoid undefined correlation when all digits are identical
            per_bag_corrs.append(0.0)
            continue

        corr, _ = spearmanr(w, digits)
        if np.isnan(corr):
            corr = 0.0
        per_bag_corrs.append(corr)

    mean_corr = np.mean(per_bag_corrs)
    return mean_corr

### 1. Create Fragment dataset

In Multiple Instance Learning (MIL), each bag is a collection of instances (here, molecular fragments), and the task often involves predicting a label for the entire bag. In this specific setup:

* Each molecule is broken into BRICS fragments.

* A fixed number of these fragments are randomly sampled to form a bag.

* A chemical property is calculated for each fragment.

* The sum of fragment properties becomes the bag's label.

* The function returns: The fragment bags (as RDKit Mol objects), The bag labels (total property), The individual fragment properties (per bag).

**The list of properties that can be calculated for fragments:**

* ``LogP`` – Lipophilicity

* ``MolWt`` – Molecular weight

* ``TPSA`` – Topological polar surface area

* ``NumHDonors`` – Number of hydrogen bond donors

* ``NumHAcceptors`` – Number of hydrogen bond acceptors

* ``MolMR`` – Molar refractivity

* ``NumRotatableBonds`` – Flexibility

* ``RingCount`` – Number of rings

* ``FractionCSP3`` – Fraction of sp³-hybridized carbon atoms

In [ ]:
REPO_ID = "KagakuData/notebooks"

csv_path = hf_hub_download(REPO_ID, filename="chembl/CHEMBL279.csv", repo_type="dataset")
data = pd.read_csv(csv_path, header=None)

smiles = list(data[0])
mols = [Chem.MolFromSmiles(s) for s in smiles]

In [ ]:
bag_size = 5
property_name = "LogP"

# create dataset
bags, labels, contribs = create_fragment_bags(mols, bag_size=bag_size, property_name=property_name)

In [ ]:
print(f"Total number of molecules: {len(mols)}")
print(f"Total number of bags (successfully generated fragments): {len(bags)}")

In [ ]:
bags[0]

In [ ]:
contribs[0]

### 2. Calculate fragment descriptors

In [ ]:
def compute_fragment_descriptors(bags, n_bits=128, radius=2):
    bags_descriptors = []
    for bag in bags:
        descs = [np.array(AllChem.GetMorganFingerprintAsBitVect(frag, radius, nBits=n_bits))
                 for frag in bag]
        bags_descriptors.append(descs)
    return bags_descriptors

In [ ]:
dsc = compute_fragment_descriptors(bags)

### 3. Build model

In [ ]:
x_train, x_test, y_train, y_test, key_train, key_test, frg_train, frg_test = train_test_split(dsc, labels, contribs, bags, random_state=42)
#
scaler = BagMinMaxScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
model = DynamicPoolingNetworkRegressor()
model.hopt(x_train_scaled, y_train, param_grid=DEFAULT_PARAM_GRID, verbose=True)
model.fit(x_train_scaled, y_train)

In [ ]:
y_pred = model.predict(x_test_scaled)
w_pred = model.get_instance_weights(x_test_scaled)
w_pred = [w.flatten() for w in w_pred]

In [ ]:
print(f"Regression R2: {r2_score(y_test, y_pred):.2f}")
print(f"KID ranking accuracy: {kid_ranking_accuracy(key_test, w_pred):.2f}")

In [ ]:
N = 1
display_fragments_with_weights(frg_test[N], key_test[N], w_pred[N], sort=True, max_fragments=10,
                               title=f"Bag {N}\nPredicted label:{y_pred[N].item():.2f}\nTrue label: {y_test[N]:.2f}")

### 4. Mini-benchmark

In [ ]:
regressor_list = [
    
        # attention mil networks
        ("AdditiveAttentionNetworkRegressor", AdditiveAttentionNetworkRegressor()),
        ("SelfAttentionNetworkRegressor", SelfAttentionNetworkRegressor()),
        ("HopfieldAttentionNetworkRegressor", HopfieldAttentionNetworkRegressor()),

        # other mil networks
        ("DynamicPoolingNetworkRegressor", DynamicPoolingNetworkRegressor()),
    ]

In [ ]:
bag_size = 5

res_df = pd.DataFrame()
tasks = ["LogP", "MolWt", "TPSA", "NumHDonors", "NumHAcceptors", 
         "MolMR", "NumRotatableBonds", "RingCount", "FractionCSP3"]

for task_idx, task in enumerate(tasks, 1):
    print(f"\n[Task {task_idx}/{len(tasks)}] Starting task: '{task}'")

    # create data
    smiles = list(pd.read_csv("data/CHEMBL1824/train.csv", header=None)[0])
    mols = [Chem.MolFromSmiles(s) for s in smiles]
    
    bags, labels, contribs = create_fragment_bags(mols, bag_size=bag_size, property_name=task)
    dsc = compute_fragment_descriptors(bags)
    
    x_train, x_test, y_train, y_test, key_train, key_test, frg_train, frg_test = train_test_split(
        dsc, labels, contribs, bags, random_state=42)
    
    # scale features
    scaler = BagMinMaxScaler()
    scaler.fit(x_train)
    x_train_scaled, x_test_scaled = scaler.transform(x_train), scaler.transform(x_test)
    
    # build models
    for model_idx, (name, model) in enumerate(regressor_list, 1):
        print(f"  [Model {model_idx}/{len(regressor_list)}] Training model: '{name}'")

        # train model
        # model.hopt(x_train_scaled, y_train, param_grid=DEFAULT_PARAM_GRID, verbose=True)
        model.fit(x_train_scaled, y_train)
        
        # predict
        y_pred = model.predict(x_test_scaled)
        w_pred = model.get_instance_weights(x_test_scaled)
        w_pred = [w.flatten() for w in w_pred]
        #
        res_df.loc[name, f"PRED_{task}"] = r2_score(y_test, y_pred)
        res_df.loc[name, f"KID_{task}"] = kid_ranking_accuracy(key_test, w_pred)

print("\nAll tasks and models completed.")

In [ ]:
res_df.round(2)